#  AutoGrad & Optimizer

논문 구현 : 수 많은 반복의 연속  
Layer = Block 

예시) Transformer
- softmax
- Linear(xW+b)
- Normalization
- Multi-Head Attension
- Layer를 훔쳐서 큰 block을 만드는 Encoder, Decoder  

즉 Layer를 쌓는것은 블록의 연속

이 것을 만들기 위한 기본이 Pytorch의 nn.moudle

### torch.nn.Module
- 딥러닝을 구성하는 Layer의 base class(Auto Grad)
- 4가지를 정함 : Input, Output, Forward(이 때 무슨일이 일어나는가), Backward(weight)
- 학습의 대상이 되는 parameter(tensor) 정의

<img src = "../images/ai_33.png">

가장 일반적인 nn.module
- Forwardpass : input(x,y)이 있고 output(z)이 있다.  
- Backwardpass : 미분 값을 넣어 각각의 미분값 뱉음

### nn.Parameter
- Tensor 객체의 상속 객체
- nn.Module 내에서 __attribute__ 가 될 때는 required_grad=True(Auto Grad)로 지정되어 학습 대상이 되는 Tensor
- 직접 지정할 일은 잘 없다 : 대부분의 layer에는 weights 값들이 지정되어 잇다.

In [20]:
import torch
from torch import Tensor
from torch import nn

# 예시(torch에는 이미 Linear가 구현되어 있다)
class MyLiner(nn.Module):
    def __init__(self, in_features, out_features, bias = True):
        super().__init__()
        self.in_features = in_features 
        self.out_features = out_features
        
        self.weights = nn.Parameter(
                torch.randn(in_features, out_features))
        
        self.bias = nn.Parameter(torch.randn(out_features))
    def forward(self, x : Tensor):
        return x @ self.weights + self.bias

In [21]:
x = torch.randn(5, 7)

In [26]:
x.size()

torch.Size([5, 7])

In [25]:
layer = MyLiner(7, 12) # x의 크기를 변경해주는 class MyLiner
layer(x).size()

torch.Size([5, 12])

In [27]:
for value in layer.parameters(): # 미분의 대상을 보여준다.
    print(value) # backward가 일어날 때 autograd가 됨

Parameter containing:
tensor([[-1.4592e-01,  5.0993e-01, -2.3244e-01, -7.3565e-02,  3.9677e-02,
          2.2398e-01,  2.0214e-01, -2.5409e+00, -1.9219e+00, -1.7530e-01,
          6.3914e-01, -1.6434e+00],
        [ 1.7428e-01,  1.0911e+00,  6.3016e-01,  2.4145e+00,  8.7708e-01,
         -3.5512e-01,  1.1642e+00, -1.4017e+00,  1.4199e-01,  1.3938e+00,
         -5.7261e-01, -8.3358e-01],
        [ 3.7805e-01,  8.3586e-01, -1.3863e+00, -4.6463e-01,  9.0152e-01,
          8.4283e-01, -2.1672e-01,  1.2465e+00, -7.9584e-03, -6.8982e-01,
         -1.6935e-01, -1.0023e+00],
        [ 3.6371e-01, -1.4043e+00, -3.1261e-01, -1.8607e-01, -1.0783e+00,
          1.4753e+00, -2.3888e+00,  4.6925e-01, -1.7411e+00, -3.6349e-02,
         -8.7827e-01,  6.0790e-01],
        [ 1.1703e+00, -8.2422e-01,  1.3809e+00,  5.0822e-01,  5.9645e-01,
          5.3550e-01, -6.7792e-01,  1.2961e+00,  1.7554e+00,  9.6315e-01,
          3.2037e-02, -2.0908e+00],
        [ 4.6476e-01, -6.7218e-01,  9.8372e-01, -5.1118e-0

### Backward
- Layer에 있는 Parameter 들의 미분을 수행
- Forward의 결과값(model의 __output=예측치__)과 실제값간의 loss에 대해 미분
- 해당 값으로 Parameter 업데이트
- 총 4단계
 1. optimizer.zero_grad() 
 2. loss = criterion(outputs, labels)
 3. loss.backward()
 4. optimizer.step()

$$y = 2x + 1  $$

In [40]:
import numpy as np
# train data 생성
x_values = [i for i in range(11)]
x_train = np.array(x_values, dtype = np.float32)
x_train = x_train.reshape(-1, 1)

y_values = [2*i + 1 for i in x_values]
y_train = np.array(y_values, dtype = np.float32)
y_train = y_train.reshape(-1, 1)

x_train

array([[ 0.],
       [ 1.],
       [ 2.],
       [ 3.],
       [ 4.],
       [ 5.],
       [ 6.],
       [ 7.],
       [ 8.],
       [ 9.],
       [10.]], dtype=float32)

In [41]:
y_train

array([[ 1.],
       [ 3.],
       [ 5.],
       [ 7.],
       [ 9.],
       [11.],
       [13.],
       [15.],
       [17.],
       [19.],
       [21.]], dtype=float32)

In [46]:
import torch
from torch.autograd import Variable
class LinearRegression(torch.nn.Module):
    def __init__(self, inputSize, outputSize):
        super(LinearRegression, self).__init__()
        self.linear = torch.nn.Linear(inputSize, outputSize)
    
    def forward(self, x):
        out = self.linear(x)
        return out

In [53]:
inputDim = 1 
outputDim = 1
learningRate = 0.01
epochs = 1000

model = LinearRegression(inputDim, outputDim)

criterion = torch.nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr = learningRate)

In [54]:
for epoch in range(epochs):
    inputs = Variable(torch.from_numpy(x_train))
    labels = Variable(torch.from_numpy(y_train))

    optimizer.zero_grad()

    outputs = model(inputs)

    loss = criterion(outputs, labels)
    print(loss)

    loss.backward()

    optimizer.step()

#     print(epoch, loss.item())

tensor(250.6362, grad_fn=<MseLossBackward0>)
0 250.6361846923828
tensor(20.5625, grad_fn=<MseLossBackward0>)
1 20.5625
tensor(1.7948, grad_fn=<MseLossBackward0>)
2 1.7947888374328613
tensor(0.2627, grad_fn=<MseLossBackward0>)
3 0.2626533508300781
tensor(0.1364, grad_fn=<MseLossBackward0>)
4 0.13638392090797424
tensor(0.1248, grad_fn=<MseLossBackward0>)
5 0.12480084598064423
tensor(0.1226, grad_fn=<MseLossBackward0>)
6 0.12258653342723846
tensor(0.1212, grad_fn=<MseLossBackward0>)
7 0.12115068733692169
tensor(0.1198, grad_fn=<MseLossBackward0>)
8 0.11979236453771591
tensor(0.1185, grad_fn=<MseLossBackward0>)
9 0.1184542253613472
tensor(0.1171, grad_fn=<MseLossBackward0>)
10 0.11713140457868576
tensor(0.1158, grad_fn=<MseLossBackward0>)
11 0.11582336574792862
tensor(0.1145, grad_fn=<MseLossBackward0>)
12 0.11452999711036682
tensor(0.1133, grad_fn=<MseLossBackward0>)
13 0.11325094848871231
tensor(0.1120, grad_fn=<MseLossBackward0>)
14 0.11198630928993225
tensor(0.1107, grad_fn=<MseLossBac

tensor(0.0004, grad_fn=<MseLossBackward0>)
517 0.00039450309122912586
tensor(0.0004, grad_fn=<MseLossBackward0>)
518 0.00039009356987662613
tensor(0.0004, grad_fn=<MseLossBackward0>)
519 0.0003857315459754318
tensor(0.0004, grad_fn=<MseLossBackward0>)
520 0.0003814292431343347
tensor(0.0004, grad_fn=<MseLossBackward0>)
521 0.0003771660849452019
tensor(0.0004, grad_fn=<MseLossBackward0>)
522 0.000372959126252681
tensor(0.0004, grad_fn=<MseLossBackward0>)
523 0.00036879556137137115
tensor(0.0004, grad_fn=<MseLossBackward0>)
524 0.00036467835889197886
tensor(0.0004, grad_fn=<MseLossBackward0>)
525 0.0003606077516451478
tensor(0.0004, grad_fn=<MseLossBackward0>)
526 0.00035657521220855415
tensor(0.0004, grad_fn=<MseLossBackward0>)
527 0.00035258749267086387
tensor(0.0003, grad_fn=<MseLossBackward0>)
528 0.00034866423811763525
tensor(0.0003, grad_fn=<MseLossBackward0>)
529 0.00034476155997253954
tensor(0.0003, grad_fn=<MseLossBackward0>)
530 0.00034091711859218776
tensor(0.0003, grad_fn=<Ms

In [55]:
with torch.no_grad():
    predicted = model(Variable(torch.from_numpy(x_train))).data.numpy()
    print(predicted)

[[ 0.99753255]
 [ 2.997888  ]
 [ 4.9982433 ]
 [ 6.9985986 ]
 [ 8.998955  ]
 [10.9993105 ]
 [12.999665  ]
 [15.000021  ]
 [17.000376  ]
 [19.00073   ]
 [21.001087  ]]


In [56]:
y_train

array([[ 1.],
       [ 3.],
       [ 5.],
       [ 7.],
       [ 9.],
       [11.],
       [13.],
       [15.],
       [17.],
       [19.],
       [21.]], dtype=float32)

### Backward from the scratch
- 실제 backward는 Module 단계에서 직접 지정가능(하지만 할필요가 x)
- Module에서 backward와 optimizer 오버라이딩
- 사용자가 직접 미분 수식을 써야하는 부담감  
    $\rightarrow$ 쓸일은 없으나 순서는 이해할 필요 있음

예시) [로지스틱 회귀 (sigmoid function)](https://colab.research.google.com/drive/1MJF1S8EcRlSjRGHF4XklgMUuMqTQpZHT#scrollTo=pJH8jaQil5gG)

$$ h_\theta(x) = \frac{1}{1 + e^{z}} $$

$$ z = -\theta^{T} x$$

# PyTorch Dataset

- 모델에 데이터를 먹이는 법

<img src = "../images/ai_32.png">

1. Data가 있는 폴더가 있다.
2. Data set class에서 를 어떻게 시작(init)할건지 길이가 얼만지(len) 어떻게 불러올것인지(getitem > __map-style__ 중요)
3. transforms에서 data 변형(tensor로)
4. DataLoader : 데이터를 묶어서 model에 feeding해줌


### Data set 클래스
- 데이터 입력 형태를 정의하는 클래스
- 데이터를 입력하는 방식의 표준화
- Image, Text, Audio등에 따른 입력 정의 

In [79]:
import torch
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
    # 초기 데이터 생성 방법 지정
    def __init__(self, text, labels):
        self.labels = labels
        self.data = text
    
    # 데이터의 전체 길이
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        label = self.labels[idx]
        text = self.data[idx]
        sample = {"Text" : text, "Class" : label}
        return sample

In [80]:
text = ['Happy', 'Amazing', 'Sad', 'Unhapy', 'Glum']
labels = ['Positive', 'Positive', 'Negative', 'Negative', 'Negative']
MyDataset = CustomDataset(text, labels)

In [81]:
type(MyDataset)

__main__.CustomDataset

### Dataset 클래스 생성시 유의점
- 데이터 형태에 따라 각 함수를 다르게 정의함
- 모든 것을 데이터 생성 시점에 처리할 필요는 없음 : image의 Tensor 변화는 학습에 필요한 시점에 변환
- Dataset에 대한 표준화된 처리방법 제공 필요 
- 최근엔 HuggingFace등 표준화된 라이브러리 사용

### DataLoader 클래스
- Data의 Batch를 생성해주는 클래스
- 학습직전(GPU feed전) 데이터의 변환을 책임
- Tensor로 변환 + Batch 처리가 메인 업무
- 병렬적인 데이터 전처리 코드의 고민 필요

#### 기억해야할 DataLoader의 parameters
- sampler, batch_sampler : 데이터를 어떻게 뽑을지 정함
- collate_fn : data가 비어잇거나 크기가 다른 Variable을 0으로 채워줌


In [91]:
MyDataLoader = DataLoader(MyDataset, batch_size=2, shuffle=True)
next(iter(MyDataLoader))

{'Text': ['Amazing', 'Glum'], 'Class': ['Positive', 'Negative']}

In [92]:
MyDataLoader = DataLoader(MyDataset, batch_size=3, shuffle=True)
for dataset in MyDataLoader:
    print(dataset)

{'Text': ['Unhapy', 'Sad', 'Amazing'], 'Class': ['Negative', 'Negative', 'Positive']}
{'Text': ['Happy', 'Glum'], 'Class': ['Positive', 'Negative']}


### Casestudy
- [실습해보기](https://colab.research.google.com/drive/1Lly2TGJ-L146AAOoQZMTa7S8Waw4ejor#scrollTo=a17b9074)
- vision data다운을 받을 수 있음, 직접 생성해보는것 권장
- 데이터 다운로드 부터 loader까지 구현해보기
- NotMNIST 데이터의 다운로드 자동화 도전|